In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import numpy as np
import pandas as pd
from tqdm.notebook import trange, tqdm, tqdm_notebook
import requests

In [4]:
driver = webdriver.Firefox()

In [5]:
driver.get("https://www.marjane.ma/product/1-courses-en-ligne")
time.sleep(10)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [15]:
s_list_level_1=soup.findAll("li",{"class":"jsx-1782791432 jsx-392234169"})
list_level_1=[]
for i in range(len(s_list_level_1)):
    list_level_1.append("https://www.marjane.ma"+s_list_level_1[i].find("a")["href"])

In [52]:
list_level_2=[]
for i in tqdm(range(len(list_level_1))):
    driver.get(list_level_1[i])
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.jsx-1407372038:nth-child(1)")))
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    
    level_2_amount=len(soup.find("div",{"class":"mobile-categories-list"}).findAll("button"))-1
    
    for l2a in range(level_2_amount):
        l2aa=str(l2a+1)
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "ul.jsx-1407372038:nth-child(3) > li:nth-child("+l2aa+")"))).click()
        time.sleep(10)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        s_list_level_2=soup.find("ul",{"class":"deployed-list"}).findAll("li")
        
        
        for l2 in range(len(s_list_level_2)):
            list_level_2.append("https://www.marjane.ma"+s_list_level_2[l2].find("a")["href"])

  0%|          | 0/19 [00:00<?, ?it/s]

In [131]:
def keep_scrolling_to_the_bottom():
    while True:
        previous_scrollY = driver.execute_script( 'return window.scrollY' )
        driver.execute_script( 'window.scrollBy( 0, 300 )' )
        time.sleep( 0.5 )
        if previous_scrollY == driver.execute_script( 'return window.scrollY' ):
            break

In [177]:
#empty data frame at first
df=pd.read_csv("marjane.csv")
product_n=[]
lv3=[]
pn=[]
l1=[]
l2=[]
l3=[]

product_url=[]
product_price=[]
product_price_sub=[]
product_price_fidelite=[]
product_name=[]

for i in tqdm(range(len(list_level_2))):
    if list_level_2[i] in df["pn"].unique():
        pass
    else:
        driver.get(list_level_2[i])
        lv3.append(list_level_2[i])
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.jsx-864416106:nth-child(1) > div:nth-child(1) > a:nth-child(1)")))   
        keep_scrolling_to_the_bottom()

        time.sleep(5)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        product_n.append(int(soup.find("p",{"class":"jsx-3735054823"}).text.split(" ")[0]))
        try:
            l1s=soup.find("ul",{"class":"jsx-508879282"}).findAll("li")[1].text.replace("\xa0"," ").replace("- "," ").replace(" -"," ").strip()
            l2s=soup.find("ul",{"class":"jsx-508879282"}).findAll("li")[2].text.replace("\xa0"," ").replace("- "," ").replace(" -"," ").strip()
            l3s=soup.find("ul",{"class":"jsx-508879282"}).findAll("li")[3].text.replace("\xa0"," ").replace("- "," ").replace(" -"," ").strip()
        except:
            l1s=np.nan
            l2s=np.nan
            l3s=np.nan

        product_list=soup.findAll("li",{"class":"jsx-864416106"})
        
        pn=[]
        l1=[]
        l2=[]
        l3=[]

        product_url=[]
        product_price=[]
        product_price_sub=[]
        product_price_fidelite=[]
        product_name=[]
        for p in range(len(product_list)):
            pn.append(list_level_2[i])
            l1.append(l1s)
            l2.append(l2s)
            l3.append(l3s)

            product_url.append("https://www.marjane.ma"+product_list[p].find("a")["href"])
            product_price.append(product_list[p].find("span",{"class":"price"}).text)

            try:
                product_price_fidelite.append(product_list[p].find("span",{"class":"fidelity-price"}).text)
            except:
                product_price_fidelite.append(np.nan)

            try:
                product_price_sub.append(product_list[p].find("span",{"class":"sub-price"}).text)
            except:
                product_price_sub.append(np.nan)            

            product_name.append(product_list[p].find("h2").text)

        ndf=pd.DataFrame({"product_url":product_url,"product_name":product_name,"product_price":product_price,
                  "product_price_sub":product_price_sub,"product_price_fidelite":product_price_fidelite,
                 "category":l1,"sub_category":l2,"sub_sub_category":l3,
                      "pn":pn})
        df=pd.concat([df,ndf]).reset_index(drop=True)
        df.to_csv("marjane.csv", encoding="utf-8-sig",index=False)

  0%|          | 0/517 [00:00<?, ?it/s]

In [182]:
df["product_price"]=df["product_price"].apply(lambda x:float(x.split(" ")[0].replace(",",".")))

pps=[]
for i in range(len(df)):
    if pd.isna(df["product_price_sub"].iloc[i]):
        pps.append(np.nan)
    else:
        pps.append(float(df["product_price_sub"].iloc[i].split(" ")[0].replace(",",".")))
df["product_price_sub"]=pps

ppf=[]
for i in range(len(df)):
    if pd.isna(df["product_price_fidelite"].iloc[i]):
        ppf.append(np.nan)
    else:
        ppf.append(float(df["product_price_fidelite"].iloc[i].split(" ")[0].replace(",",".")))
df["product_price_fidelite"]=ppf  

df=df[pd.notna(df["category"])].reset_index(drop=True)
df=df[df["sub_sub_category"].apply(lambda x:"Voir tous les produits" not in x)].reset_index(drop=True)

In [212]:
df.to_csv("marjane.csv", encoding="utf-8-sig",index=False)

In [214]:
df.to_excel("marjane.xlsx", encoding="utf-8-sig",index=False)

C:\Users\Jsh\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
